In [ ]:
import ifcopenshell
from utils.JupyterIFCRenderer import JupyterIFCRenderer

# Load the IFC model
ifc_file_path = "../data/04_Flat.ifc"
ifc_model = ifcopenshell.open(ifc_file_path)

# The Viewer Component
viewer = JupyterIFCRenderer(ifc_model, size=(800, 600))
viewer.setDefaultColors()

# Function to handle viewer selection
def on_selection(selected_product, event):
    return selected_product

# Assign the selection handler function to the viewer's selection_observer attribute
viewer.selection_observer = on_selection

# Display the viewer
viewer


In [ ]:
from rdflib import Graph, URIRef, RDF
from rdflib.plugins.sparql import prepareQuery

# Load RDF data from a file
rdf_file_path = "00_OpenSmartHomeData.ttl"
g = Graph()
g.parse(rdf_file_path, format="turtle")

# Dictionary to store the mapping of room URIs to human-readable names
room_names = {
    "https://w3id.org/ibp/osh/OpenSmartHomeDataSet#Kitchen": "Kitchen",
    "https://w3id.org/ibp/osh/OpenSmartHomeDataSet#Room1": "Room 1",
    "https://w3id.org/ibp/osh/OpenSmartHomeDataSet#Room2": "Room 2",
    "https://w3id.org/ibp/osh/OpenSmartHomeDataSet#Room3": "Room 3",
    "https://w3id.org/ibp/osh/OpenSmartHomeDataSet#Bathroom": "Bathroom",
    "https://w3id.org/ibp/osh/OpenSmartHomeDataSet#Toilet": "Toilet",
    "https://w3id.org/ibp/osh/OpenSmartHomeDataSet#Küche": "Kitchen",
    "https://w3id.org/ibp/osh/OpenSmartHomeDataSet#Bad": "Bathroom",
    # Add more room URIs and names as needed
}

# Dictionary to store the mapping of rooms to sensor types
room_sensor_mapping = {}

# SPARQL query to extract information about rooms and their sensors
query = prepareQuery("""
    SELECT ?room ?sensor_type WHERE {
        ?room <https://w3id.org/bot#containsElement> ?sensor .
        ?sensor a ?sensor_type .
    }
""")

# Execute the SPARQL query and process the results
for row in g.query(query):
    room_uri, sensor_type = row
    room_uri = str(room_uri)
    room_name = room_names.get(room_uri, room_uri)  # Get the human-readable name, or use the URI if not found in the dictionary
    sensor_type = str(sensor_type)
    if room_name not in room_sensor_mapping:
        room_sensor_mapping[room_name] = {sensor_type}
    else:
        room_sensor_mapping[room_name].add(sensor_type)

# Function to extract room name from the entity string
def get_room_name(entity_string):
    room_name = entity_string.split(",")[7].strip().strip("'")
    if room_name.startswith("Raum "):
        room_number = room_name.split(" ")[1]
        room_name = f"Room {room_number}"
    elif room_name == "Küche":
        room_name = "Kitchen"
    elif room_name == "Bad":
        room_name = "Bathroom"
    return room_name

# Function to handle selected room and display associated sensors
def process_selection(selected_product):
    if selected_product is not None:
        room_name = get_room_name(str(selected_product))
        if room_name in room_sensor_mapping:
            sensors = room_sensor_mapping.get(room_name, set())
            print(f"Selected Space: {room_name}")
            if sensors:
                for sensor in sensors:
                    print(f"- Sensor type: {sensor}")
            else:
                print("No sensor information available for this room.")
        else:
            print(f"Selected Entity: {selected_product}")

# Replace this with the selected product obtained from the first code snippet
selected_product = viewer.getSelectedProduct()

# Process the selected room and display associated sensors or selected entity
process_selection(selected_product)
